In [9]:
import sys
sys.path.append('..\\..')
import gdspy
import numpy as np
from importlib import reload
from copy import deepcopy
from QCreator import elements
from QCreator import general_sample_creator as creator
from QCreator import meshing
import QCreator.auxiliary_functions as auxfuncs
from matplotlib import pyplot as plt
reload(gdspy)

<module 'gdspy' from 'C:\\Users\\iliab\\anaconda3\\lib\\site-packages\\gdspy\\__init__.py'>

In [10]:
tl_core = 11.
tl_gap = 6.
tl_ground = 20.

In [11]:
pad_offset = 1100
pad_element_offset = 400



layers_configuration = {
    'total':0,
    'restricted area':10,
    'for removing':100,
    'JJs':3,
    'air bridges':2,
    'air bridge pads':1,
    'air bridge sm pads':4,
    'vertical gridlines':15,
    'horizontal gridlines':16,
    'inverted':17,
    'bandages':101
}

sample = creator.Sample('Res-test-16-bridges',layers_configuration, epsilon=11.45)


airbridge = elements.AirBridgeGeometry(pad_width = 30,
                                       pad_length = 30,
                                       narrow_width = 10,
                                       narrow_length = 56,
                                       pad_distance = 36,
                                       sm_pad_length = 10,
                                       sm_pad_distance = 56,
                                       layer_configuration = sample.layer_configuration,
                                       bridge_style='misis')

# airbridge = elements.AirBridgeGeometry(pad_width = 36,
#                                        pad_length = 22,
#                                        pad_distance = 62,
#                                        narrow_width = 20,
#                                        narrow_length = 46,
#                                        sm_pad_length = 10,
#                                        sm_pad_distance = 70,
#                                        layer_configuration = sample.layer_configuration)

In [12]:
#specify sample vertical and horizontal lengths
sample.chip_geometry.sample_vertical_size=4.7e3
sample.chip_geometry.sample_horizontal_size=9.7e3
central_line_y = sample.chip_geometry.sample_vertical_size/2
chip_edge_ground = elements.ChipEdgeGround(sample.chip_geometry, sample.layer_configuration, sample.pads, 800)
sample.add(chip_edge_ground)

# grid_ground = elements.GridGround(sample.chip_geometry, sample.layer_configuration, sample.objects, 
#                                   width=4, period=25)
# sample.add(grid_ground)

In [13]:
# 1. Create contact pads:
pads_left = []
pads_right = []
for pad_side_id in range(1):
    pad = elements.Pad('pad-left-' + str(pad_side_id),
                       (pad_offset, sample.chip_geometry.sample_vertical_size / 2), np.pi, tl_core,
                       tl_gap, tl_ground,
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry,
                       **elements.default_pad_geometry())
    pads_left.append(pad)
    sample.add(pad)
    pad = elements.Pad('pad-right-' + str(pad_side_id),
                       (sample.chip_geometry.sample_horizontal_size - pad_offset,
                        sample.chip_geometry.sample_vertical_size / 2), 0, tl_core,
                       tl_gap, tl_ground,
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry,
                       **elements.default_pad_geometry())
    pads_right.append(pad)
    sample.add(pad)

pad0 = pads_left[0]
pad1 = pads_right[0]

In [14]:
initial_spacing = 200

coupler_lengths = [280, 300, 330, 365, 390, 420, 460, 520]
coupler_starts = [[pad_offset + initial_spacing,
                  sample.chip_geometry.sample_vertical_size / 2]]
for i in range(1, 8):
    coupler_starts.append([pad_offset + i * (sample.chip_geometry.sample_horizontal_size - 2*pad_offset)/8,
                           sample.chip_geometry.sample_vertical_size / 2])
    
closed_end_meander_lengths1 = [2900, 3110, 3400, 3680, 4000, 4450, 5000, 5600]
closed_end_meander_lengths2 = [3000, 3250, 3600, 3850, 4250, 4730, 5300, 6000]
length_lefts1 = [150, 200, 200, 200, 200, 200, 300, 400]
length_rights1 = [150, 200, 200, 200, 200, 350, 150, 150]
length_lefts2 = [150, 200, 200, 200, 200, 350, 150, 150]
length_rights2 = [150, 200, 200, 200, 200, 200, 300, 400]
    
object_left = pad0
port_left = 'port'
    
for resonator_pair_id in range(8):
    g1, g2, par1, par2, res12 = auxfuncs.draw_double_resonator(
        sample,
        coupler_starts[resonator_pair_id][0],
        coupler_starts[resonator_pair_id][1],
        coupler_lengths[resonator_pair_id],
        resonator_core = 7,
        resonator_gap = 8,
        resonator_ground = 20,
        tl_core = 11,
        tl_gap = 6,
        tl_ground = 20,
        grounding_width = 10,
        closed_end_meander_length1 = closed_end_meander_lengths1[resonator_pair_id],
        length_left1 = length_lefts1[resonator_pair_id],
        length_right1 = length_rights1[resonator_pair_id],
        closed_end_meander_length2 = closed_end_meander_lengths2[resonator_pair_id],
        length_left2 = length_lefts2[resonator_pair_id],
        length_right2 = length_rights2[resonator_pair_id],
        open_end_length1 = 500,
        open_end_length2 = 500,
        port_orientation='left',
        airbridge=airbridge,
        min_bridge_spacing_closed_end=100,
        min_bridge_spacing_open_end=150,
        meander_r=55)

    sample.connect_cpw(object_left, g1, port1=port_left, port2='narrow', name='right TL {}'.format(resonator_pair_id), 
                       points=[], airbridge=airbridge, min_spacing=150)
    object_left = g2
    port_left = 'narrow'
    

sample.connect_cpw(object_left, pad1, port1=port_left, port2='port', name='right TL', points=[],
                   airbridge=airbridge, min_spacing=150)

[CPW "right TLs0", l=81.125,
 CPW "right TLs1", l=162.250,
 CPW "right TLs2", l=81.125]

In [15]:
sample.draw_design()
sample.watch()

In [16]:
sys, connections, elements_ = sample.get_tls(cutoff=2e11)
#f, delta, modes = sys.get_modes()
#num_modes=10

#(res_fr,kappa,_)=auxfuncs.search_for_resonators_qubits(f,delta,0,15)[:num_modes]

In [52]:
sys.create_dynamic_equation_matrices()[0].shape

(6304, 6304)

In [ ]:
f, delta, modes = sys.get_modes()

In [ ]:
num_modes=16
(res_fr,kappa,_)=auxfuncs.search_for_resonators_qubits(f,delta,0,10)[:num_modes]
#f[:num_modes]/(2e6*np.pi), delta[:num_modes]/(2e6*np.pi), f/(2*np.pi*2*delta)

In [ ]:
f_ = np.linspace(3e9, 10e9, 1601)

In [ ]:
s21 = sample.get_s21('pad-left-0', 'pad-right-0', f_)
#plt.plot(f_, s21)


In [ ]:
plt.plot(f_, abs(s21))

In [53]:
sample.write_to_gds()

Gds file has been writen here:  C:\Users\avasi\Documents\GitHub\QCreator\designs\Resonators_test/Res-test-16-bridges.gds


In [ ]:
from math import sqrt
l = 3400e-6
f0 = (3e8/sqrt(11.45)) * 1/(4*l)
f0/1e9

In [ ]:
f_bridge = [
    4.1424458,
    4.59551078,
    4.88016854,
    5.51433467,
    5.94025965,
    6.90750097,
    7.1425325,
    7.69999696,
    7.9779454,
    8.68035539
]

f_nobridge = [
    4.18101758,
    4.63208409,
    4.92084733,
    5.55735695,
    5.99094905,
    6.96159938,
    7.19772259,
    7.75222642,
    8.04561973,
    8.7448298
]

numbers = np.arange(10)

lengths = [
    3350,
    3650,
    3790,
    4090,
    4230,
    4930,
    5320,
    6020,
    6400,
    7100
]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)


fig, ax = plt.subplots(figsize=(10, 8))
ax.set_xlabel("№", fontsize=14)
ax.set_ylabel("frequency", fontsize=14)
ax.grid(which="major", linewidth=1.2)
ax.grid(which="minor", linestyle="--", color="gray", linewidth=0.5)
ax.scatter(numbers,f_bridge, c = 'red', label = 'bridges')
ax.scatter(numbers,f_nobridge, c = 'blue', label = 'w/o bridges')
ax.legend()

#ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='major', length=10, width=2)
ax.tick_params(which='minor', length=5, width=1)
plt.savefig('demo.png', transparent=True)

In [ ]:
diff = np.empty(len(f_bridge))
diff = np.array(f_nobridge) - np.array(f_bridge)
diff

In [ ]:
plt.plot(numbers, diff)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_xlabel("№", fontsize=14)
ax.set_ylabel("frequency", fontsize=14)
ax.grid(which="major", linewidth=1.2)
ax.grid(which="minor", linestyle="--", color="gray", linewidth=0.5)
ax.scatter(numbers,diff, c = 'red', label = 'difference')
#ax.scatter(numbers,f_nobridge, c = 'blue', label = 'bridges')
ax.legend()

#ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='major', length=10, width=2)
ax.tick_params(which='minor', length=5, width=1)
plt.savefig('demo1.png', transparent=True)

In [ ]:
plt.plot(lengths, diff)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_xlabel("length", fontsize=14)
ax.set_ylabel("difference", fontsize=14)
ax.grid(which="major", linewidth=1.2)
ax.grid(which="minor", linestyle="--", color="gray", linewidth=0.5)
ax.scatter(lengths,diff, c = 'red', label = 'difference')
#ax.scatter(numbers,f_nobridge, c = 'blue', label = 'bridges')
ax.legend()

#ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='major', length=10, width=2)
ax.tick_params(which='minor', length=5, width=1)
plt.savefig('demo1.png', transparent=True)

In [ ]:
import resonator_tools
from resonator_tools.circuit import notch_port, reflection_port